In [ ]:
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from tqdm.notebook import tqdm
import keras.layers

Using TensorFlow backend.


In [ ]:
!pip install -q keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def extract_features(directory):
        model = ResNet50()
        print(model.summary())
        model.layers.pop()
        model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
        print('Restructured \n')
        print(model.summary())
        #define a dictionary
        features = dict()
        for name in tqdm(os.listdir(directory)):
         filename = directory + '/' + name
         image = load_img(filename, target_size=(224, 224))
         image = img_to_array(image)
         image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
         image = preprocess_input(image)
         feature = model.predict(image, verbose=0)
         image_id = name.split('.')[0]
         features[image_id] = feature
        return features
#2048 feature vector
features = extract_features('/content/drive/My Drive/Data/Flicker8k_Dataset')
print('Extracted Features: %d' % len(features))

dump(features, open("/content/drive/My Drive/Lokesh Image Captioning/features2.pkl","wb"))

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________


Extracted Features: 8103


In [ ]:
import string

def load_doc(filename):
	
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text


def load_descriptions(doc):
	mapping = dict()
	
	for line in doc.split('\n'):
		tokens = line.split()
		if len(line) < 2:
			continue
		image_id, image_desc = tokens[0], tokens[1:]
		image_id = image_id.split('.')[0]
		image_desc = ' '.join(image_desc)
		if image_id not in mapping:
			mapping[image_id] = list()
		
		mapping[image_id].append(image_desc)
	return mapping

def clean_descriptions(descriptions):
	# translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			desc = desc.split()
			
      desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			desc = [word for word in desc if len(word)>1]
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)


def to_vocabulary(descriptions):
	# list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc


def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()


filename = '/content/drive/My Drive/Data/Flickr8k_text/Flickr8k.token.txt'

doc = load_doc(filename)
descriptions= load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

clean_descriptions(descriptions)

vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

save_descriptions(descriptions, '/content/drive/My Drive/Lokesh Image Captioning/descriptions2.txt')

Loaded: 8092 
Vocabulary Size: 8763


In [ ]:
list(descriptions.keys())[:5]

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615']

In [ ]:
descriptions['1000268201_693b08cb0e']

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [ ]:
from pickle import load

# load a pre-defined list of image identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	
	for line in doc.split('\n'):
		
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)


def load_clean_descriptions(filename, dataset):
	
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		
		tokens = line.split()
		image_id, image_desc = tokens[0], tokens[1:]
		
		if image_id in dataset:
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap descriptions
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			
			descriptions[image_id].append(desc)
	return descriptions


def load_photo_features(filename, dataset):
	
	all_features = load(open(filename, 'rb'))
	# filter features
	features={}
	for k in dataset:
		features[k]= all_features[k]
	return features


filename = '/content/drive/My Drive/Data/Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

train_descriptions = load_clean_descriptions('/content/drive/My Drive/Lokesh Image Captioning/descriptions2.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

train_features = load_photo_features('/content/drive/My Drive/Lokesh Image Captioning/features2.pkl', train)
print('Photos: train=%d' % len(train_features))
train_descriptions

Dataset: 6000
Descriptions: train=6000
Photos: train=6000


{'1000268201_693b08cb0e': ['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
  'startseq girl going into wooden building endseq',
  'startseq little girl climbing into wooden playhouse endseq',
  'startseq little girl climbing the stairs to her playhouse endseq',
  'startseq little girl in pink dress going into wooden cabin endseq'],
 '1001773457_577c3a7d70': ['startseq black dog and spotted dog are fighting endseq',
  'startseq black dog and tricolored dog playing with each other on the road endseq',
  'startseq black dog and white dog with brown spots are staring at each other in the street endseq',
  'startseq two dogs of different breeds looking at each other on the road endseq',
  'startseq two dogs on pavement moving toward each other endseq'],
 '1002674143_1b742ab4b8': ['startseq little girl covered in paint sits in front of painted rainbow with her hands in bowl endseq',
  'startseq little girl is sitting in front of large painted rainbow endse